In [1]:
#Habilitar intellisense
%config IPCompleter.greedy = True

## Aprendizaje Automático (Machine Learning)

### ¿Qué es aprender?

__[Aprendizaje Automático](https://es.wikipedia.org/wiki/Aprendizaje_autom%C3%A1tico)__


### Ejemplo de un problema de clasificación supervisado
<img src="01_spam-detection.png" style="width:600px"/>

### Variables independientes (features) y variable objetivo (response/tag/class)
<img src="02_features-input-output.png" style="width:600px"/>

<img src="03_feature.png" style="width:600px"/>
<hr\>
El proceso de "entrenamiento" busca obtener un artefacto que sea capaz de clasificar nuevas observaciones/casos. A mayor cantidad de ejemplos, se espera un mejor desempeño.
<img src="04_generalizacion.png" style="width:600px"/>

### Aprendizaje supervisado (clasificación vs regresión)
__[Algoritmos de aprendizaje supervisado](https://en.wikipedia.org/wiki/Supervised_learning#Algorithms)__

<img src="05_supervised_classification.png" style="width:600px"/>
<hr\>
<img src="06_supervised_regression.png" style="width:600px"/>

### Aprendizaje no supervisado
<img src="07_unsupervised-clustering.png" style="width:600px"/>


## Aprendizaje Automático vs Sistemas Basados en Reglas
<img src="03_ml_vs_rules.png" style="width:700px"/>

ML. Pro: adecuado (probablemente la única alternativa) en escenarios con reglas dinámicas y dependiente del contexto. Con: requiere cantidad considerable de datos históricos de buena calidad y la selección de buenos vectores de características (esto requiere experiencia y conocimiento de el funcionamiento de los algoritmos de ML)

Reglas. Pro: adecuado en escenarios con reglas relativamente estáticas o cuando no se dispone de cantidad considerable de datos históricos (en cuyo caso es la única opción). Con: pobre desempeño a la hora de capturar reglas que dependen del contexto, es muchos escencario se requiere de expertos que provean las reglas.

## Evaluación del desempeño de un clasificador (binario)

### Accuracy (Exactitud)
<img src="08_accuracy.png" style="width:700px"/>

__[Cuando la exactitud no es suficiente](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/)__

__[Matriz de Confusión](https://es.wikipedia.org/wiki/Matriz_de_confusi%C3%B3n)__


###  Precisión
<img src="09_precision.png" style="width:700px"/>

**¿Qué proporción de clasificaciones positivas fue correcta?**

###  Recall (Exhaustividad) 
<img src="10_recall.png" style="width:600px"/>


**¿Qué proporción de positivos reales se clasificó correctamente?**

###  Preguntas sobre precisión y exhaustividad

Con un clasificador (digamos un test para detectar una enfermedad) de alta exhaustividad, es más o menos probable fallar en la detección de la enfermedad?

Con un clasificador (digamos un test para detectar una enfermedad) de alta exhaustividad, es más o menos probable detectar la enfermedad en personas que no la tienen?

Si la enfermedad es mortal, quá es preferible: alta Presición o alta Exhaustividad?

## Modelo - Linea base

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
df_entrenamiento = pd.read_csv(os.path.join("procesado", "train.csv"), index_col='PassengerId')

In [4]:
df_entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
Survived              891 non-null int64
Age                   891 non-null float64
Fare                  891 non-null float64
FamilySize            891 non-null int64
IsMother              891 non-null int64
IsMale                891 non-null int64
Deck_A                891 non-null float64
Deck_B                891 non-null float64
Deck_C                891 non-null float64
Deck_D                891 non-null float64
Deck_E                891 non-null float64
Deck_F                891 non-null float64
Deck_G                891 non-null float64
Deck_Z                891 non-null float64
Pclass_1              891 non-null float64
Pclass_2              891 non-null float64
Pclass_3              891 non-null float64
Title_Lady            891 non-null float64
Title_Master          891 non-null float64
Title_Miss            891 non-null float64
Title_Mr              891 non-null float64


In [5]:
df_entrenamiento.head()

,Survived,Age,Fare,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,Deck_D,...,Title_Sir,Fare_Bin_very_low,Fare_Bin_low,Fare_Bin_high,Fare_Bin_very_high,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,22.0,7.2500,2,0,1,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1,38.0,71.2833,2,0,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,1,26.0,7.9250,1,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1,35.0,53.1000,2,0,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
5,0,35.0,8.0500,1,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


## Division Entrenamiento / Test (train - test split)

In [10]:
#Separar las caracteristicas de la variable de salida
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.ravel.html
X = df_entrenamiento.loc[:,'Age':].to_numpy().astype('float') #convertir a una matriz numpy
y = df_entrenamiento['Survived'].ravel() #convertir a un vector numpy

In [6]:
print(X.shape, y.shape)

(891, 32) (891,)


In [7]:
print(type(X), type(y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [12]:
# Ver si no se trata de un problema con clases muy "desbalanceadas"
print('media de supervivencia en el conjunto de entrenamiento : {0:.3f}'.format(np.mean(y_train)))
print('media de supervivencia en el conjunto de prueba : {0:.3f}'.format(np.mean(y_test)))

media de supervivencia en el conjunto de entrenamiento : 0.383
media de supervivencia en el conjunto de prueba : 0.385


## Clasificador linea base

In [7]:
# importar el algoritmo de clasificación
from sklearn.dummy import DummyClassifier

In [8]:
# Crear el clasificador
clasificador_lineabase = DummyClassifier(strategy='most_frequent', random_state=0)

In [13]:
# "entrenar" el clasificador
clasificador_lineabase.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [14]:
# El método "score" devuelve Accuracy (exactitud) del clasificador linea base
# Accuracy obtenido sólo con devolver la clasificación con mayor frecuencia
# Se debe superar este valor empleando Machine Learning.
print('accuracy del clasificador : {0:.2f}'.format(clasificador_lineabase.score(X_test, y_test)))

accuracy del clasificador : 0.61


In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [16]:
#La función compara las clasificaciones conocidas (del conjunto de prueba) 
# con las predicciones hechas por el clasificador para los ejemplos del conjunto de prueba
print('accuracy del clasificador : {0:.2f}'.format(accuracy_score(y_test, clasificador_lineabase.predict(X_test))))

accuracy del clasificador : 0.61


In [17]:
# matriz de confusión
print('matriz de confusión del clasificador: \n {0}'.format(confusion_matrix(y_test, clasificador_lineabase.predict(X_test))))

matriz de confusión del clasificador: 
 [[110   0]
 [ 69   0]]


In [18]:
# precision y recall
print('precision del clasificador : {0:.2f}'.format(precision_score(y_test, clasificador_lineabase.predict(X_test))))
print('recall del clasificador : {0:.2f}'.format(recall_score(y_test, clasificador_lineabase.predict(X_test))))

precision del clasificador : 0.00
recall del clasificador : 0.00


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
